In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt 
import seaborn as sns

%matplotlib inline

#warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, ShuffleSplit

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, GaussianNoise
from keras.optimizers import RMSprop
from keras import regularizers

Using TensorFlow backend.


In [2]:
#CARGAMOS LOS DATOS

traindata=pd.read_csv('Dataset_Salesforce_Predictive_Modelling_TRAIN.txt')#reading the data
testdata=pd.read_csv('Dataset_Salesforce_Predictive_Modelling_TEST.txt')
print('Number of rows and columns of the training set: ',traindata.shape)
print('Number of rows and columns of the test set: ',testdata.shape)


/home/alex/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (83) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Number of rows and columns of the training set:  (363834, 89)
Number of rows and columns of the test set:  (156315, 88)


In [3]:
#VARIABLES DE CADA TIPO

numeric_cols=traindata.select_dtypes(include=[np.number]).columns#select only numerical
nominal_cols=traindata.select_dtypes(exclude=[np.number]).columns#select only non numerical
print(numeric_cols.shape[0],'numeric columns: ',numeric_cols)
print(nominal_cols.shape[0],'nominal columns: ',nominal_cols)

87 numeric columns:  Index(['Imp_Cons_01', 'Imp_Cons_02', 'Imp_Cons_03', 'Imp_Cons_04',
       'Imp_Cons_05', 'Imp_Cons_06', 'Imp_Cons_07', 'Imp_Cons_08',
       'Imp_Cons_09', 'Imp_Cons_10', 'Imp_Cons_11', 'Imp_Cons_12',
       'Imp_Cons_13', 'Imp_Cons_14', 'Imp_Cons_15', 'Imp_Cons_16',
       'Imp_Cons_17', 'Imp_Sal_01', 'Imp_Sal_02', 'Imp_Sal_03', 'Imp_Sal_04',
       'Imp_Sal_05', 'Imp_Sal_06', 'Imp_Sal_07', 'Imp_Sal_08', 'Imp_Sal_09',
       'Imp_Sal_10', 'Imp_Sal_11', 'Imp_Sal_12', 'Imp_Sal_13', 'Imp_Sal_14',
       'Imp_Sal_15', 'Imp_Sal_16', 'Imp_Sal_17', 'Imp_Sal_18', 'Imp_Sal_19',
       'Imp_Sal_20', 'Imp_Sal_21', 'Ind_Prod_01', 'Ind_Prod_02', 'Ind_Prod_03',
       'Ind_Prod_04', 'Ind_Prod_05', 'Ind_Prod_06', 'Ind_Prod_07',
       'Ind_Prod_08', 'Ind_Prod_09', 'Ind_Prod_10', 'Ind_Prod_11',
       'Ind_Prod_12', 'Ind_Prod_13', 'Ind_Prod_14', 'Ind_Prod_15',
       'Ind_Prod_16', 'Ind_Prod_17', 'Ind_Prod_18', 'Ind_Prod_19',
       'Ind_Prod_20', 'Ind_Prod_21', 'Ind_Prod_22', 'I

In [4]:
description=traindata['Poder_Adquisitivo'].describe()
print(description)
iqr=description['75%']-description['25%']
limits=[description['25%']-1.5*iqr,description['75%']+1.5*iqr]
print('limits (lower,upper)',limits)
#NO HAY NADIE POR DEBAJO DEL LIMITE INFERIOR
print('Valores anomalos',traindata[traindata['Poder_Adquisitivo']>limits[1]].shape[0])
#SON DEMASIADOS. VOY A QUITAR SOLO A LOS MUY RICOS QUE SOLO SON 33 
traindata=traindata[traindata['Poder_Adquisitivo']<=1000000]


count    3.638340e+05
mean     1.642141e+04
std      2.436773e+04
min      3.600960e+03
25%      9.300933e+03
50%      1.292517e+04
75%      1.894854e+04
max      5.040000e+06
Name: Poder_Adquisitivo, dtype: float64
limits (lower,upper) [-5170.4740936249964, 33419.943489375]
Valores anomalos 20840


In [5]:
#EL ID DEL CLIENTE NO NOS SIRVE PARA NADA


traindata=traindata.drop('ID_Customer',axis=1) #remove the column from train
test_ids = testdata['ID_Customer'] #save id column from test
testdata=testdata.drop('ID_Customer',axis=1) #remove the column from test
nominal_cols=nominal_cols[nominal_cols!='Id'] #remove the column name from this list as well

#CONCATENAMOS AMBOS CONJUNTOS PARA APLICAR TRANSFORMACIONES

data=pd.concat([traindata,testdata],axis=0,ignore_index=True) #concatenate training and test set for future transformations
print(data['Poder_Adquisitivo'].head())#Don't worry about the SalePrice variable that is not in test. 
print(data['Poder_Adquisitivo'].tail()) #It's filled with NAs

0    19709.915000
1    37497.492167
2     4802.970000
3     8295.770000
4    24149.321667
Name: Poder_Adquisitivo, dtype: float64
520111   NaN
520112   NaN
520113   NaN
520114   NaN
520115   NaN
Name: Poder_Adquisitivo, dtype: float64


In [6]:
#VALORES NUMERICOS FALTANTES

missing_values=data.isnull().sum() #obtain the number of missing values by column
numeric_missing=missing_values[numeric_cols] #separate the numeric variables
numeric_missing['Poder_Adquisitivo']=0 #We don't want to detect SalePrice's NAs because they are not real, they just belong to test set
numeric_missing[numeric_missing>0]

#NO FALTAN VALORES NUMERICOS

Series([], dtype: int64)

In [7]:
#VALORES NOMINALES FALTANTES

nominal_missing=missing_values[nominal_cols] 
nominal_missing=nominal_missing[nominal_missing>0]
nominal_missing_df= pd.DataFrame()
nominal_missing_df[['absolute','relative']]= pd.concat([nominal_missing,nominal_missing/data.shape[0]],axis=1)
nominal_missing_df

#HABRA QUE ESTUDIAR BIEN LA COLUMNA Socio_Demo_01, DE MOMENTO ME LA CARGO PARA QUE ESTO FUNCIONE

data=data.drop(['Socio_Demo_01'],axis=1) #remove columns
nominal_cols=nominal_cols[nominal_cols!='Socio_Demo_01']


In [8]:
stdSc = StandardScaler()

numeric_cols=numeric_cols[numeric_cols!='Poder_Adquisitivo'] #We don't want to scale SalePrice

data.loc[:, numeric_cols] = stdSc.fit_transform(data.loc[:, numeric_cols])

In [9]:
#SEPARAMOS DE NUEVO LOS CONJUNTOS

traindata=data.iloc[:traindata.shape[0],:] 
testdata=data.iloc[traindata.shape[0]:,:]
testdata=testdata.drop('Poder_Adquisitivo',axis=1) #We drop the unknown variable in the test. It was just filled with NAs

X_traindata=traindata.drop('Poder_Adquisitivo',axis=1)
y_traindata=traindata['Poder_Adquisitivo']


In [14]:
batch_size = 128
epochs = 150

X_train=X_traindata.as_matrix()
y_train=y_traindata.as_matrix()

model = Sequential()
model.add( Dense( 32,activation='relu',input_dim=X_train.shape[1]))#, kernel_regularizer=regularizers.l2(0.005)))
    #model.add( Dropout(0.5) )
model.add( Dense( 32, activation='relu'))#, kernel_regularizer=regularizers.l2(0.005)) )
    #model.add( Dropout(0.25) )
model.add( Dense( 1, activation='linear') )

model.compile( loss='mse', optimizer=RMSprop(lr=0.01), metrics=['mae'] )

model.summary()
      
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=2,shuffle=True,validation_split = 0.33)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 32)                2784      
_________________________________________________________________
dense_8 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 33        
Total params: 3,873
Trainable params: 3,873
Non-trainable params: 0
_________________________________________________________________
Train on 243746 samples, validate on 120055 samples
Epoch 1/150
3s - loss: 241852833.4513 - mean_absolute_error: 6116.7596 - val_loss: 173498485.4696 - val_mean_absolute_error: 5326.2660
Epoch 2/150
3s - loss: 215807140.4960 - mean_absolute_error: 5317.5252 - val_loss: 169049179.9755 - val_mean_absolute_error: 5121.8866
Epoch 3/150
3s - loss: 211188450.8542 - mean_absolute_error: 5258.8302 - val_loss:

3s - loss: 166530698.1091 - mean_absolute_error: 4546.8581 - val_loss: 161773035.9408 - val_mean_absolute_error: 4727.2231
Epoch 57/150
3s - loss: 166385531.0384 - mean_absolute_error: 4551.0372 - val_loss: 157081604.1258 - val_mean_absolute_error: 4529.0868
Epoch 58/150
3s - loss: 166281667.3279 - mean_absolute_error: 4547.7829 - val_loss: 164495217.9285 - val_mean_absolute_error: 4597.0099
Epoch 59/150
3s - loss: 165214027.8814 - mean_absolute_error: 4548.1908 - val_loss: 157522644.8889 - val_mean_absolute_error: 4566.8617
Epoch 60/150
3s - loss: 165560526.1790 - mean_absolute_error: 4545.2495 - val_loss: 158362678.9978 - val_mean_absolute_error: 4601.9678
Epoch 61/150
3s - loss: 166085505.5340 - mean_absolute_error: 4548.2126 - val_loss: 159989928.7162 - val_mean_absolute_error: 4528.7146
Epoch 62/150
3s - loss: 166306765.2505 - mean_absolute_error: 4545.3933 - val_loss: 163961142.0500 - val_mean_absolute_error: 4646.5772
Epoch 63/150
3s - loss: 165619623.7298 - mean_absolute_error:

3s - loss: 156209972.7793 - mean_absolute_error: 4483.1224 - val_loss: 168756467.2716 - val_mean_absolute_error: 4534.5635
Epoch 118/150
3s - loss: 155026805.2438 - mean_absolute_error: 4479.7880 - val_loss: 176232739.0181 - val_mean_absolute_error: 4497.8210
Epoch 119/150
3s - loss: 155458705.9289 - mean_absolute_error: 4484.1035 - val_loss: 163466557.3636 - val_mean_absolute_error: 4547.0675
Epoch 120/150
3s - loss: 156446105.5965 - mean_absolute_error: 4485.7030 - val_loss: 174577840.0702 - val_mean_absolute_error: 4516.7171
Epoch 121/150
3s - loss: 155406603.0351 - mean_absolute_error: 4483.4092 - val_loss: 166175717.8954 - val_mean_absolute_error: 4492.0911
Epoch 122/150
3s - loss: 155707806.4908 - mean_absolute_error: 4483.9350 - val_loss: 173708406.0112 - val_mean_absolute_error: 4583.1906
Epoch 123/150
3s - loss: 154747448.4499 - mean_absolute_error: 4487.8837 - val_loss: 171849591.8462 - val_mean_absolute_error: 4559.2943
Epoch 124/150
3s - loss: 156058038.7480 - mean_absolute